# DSCI 504

## Week 2: Building the OPC Database

### Student Name: SEIF KUNGULIO

Assignment Objectives:

1. Build a database schema
2. Build database tables
3. Identify and establish Primary and Foreign Keys
4. Identify and establish key table Constraints
5. Import data into PostgreSQL
6. Query a table

**Step 1: Create a new schema named 'dsci\_504'.**

In [1]:
--Enter your SQL code here
CREATE SCHEMA dsci_504

CREATE SCHEMA

Total execution time: 00:00:00.005

**STEP 2: Create the necessary tables identified in the Assignemt 2 Table Structure document located in Canvas. Use a new code cell for each table.**

In [2]:
--Enter your SQL code here. Use a single cell for each table/task. Do not enter all tables in one cell. It will be harder to troublsehoot errors.

-- 1) Lookup: STATES
CREATE TABLE dsci_504.states (
  state_id   INT   PRIMARY KEY --The numerical state id from 1 to 50
  ,state     TEXT  NOT NULL --The two-digit state digraph
);

CREATE TABLE

Total execution time: 00:00:00.026

In [3]:
-- 2) CUSTOMERS
CREATE TABLE dsci_504.customers (
  cus_id         SERIAL PRIMARY KEY --The numerical customer ID
  ,cus_num        TEXT --A unique customer number derived from the customer name and a generated number
  ,cus_last_name  TEXT --Customer Last Name
  ,cus_first_name TEXT --Customer First Name
  ,cus_add_num    TEXT --Numerical house number
  ,cus_address    TEXT --Line 2 address
  ,cus_city       TEXT --Customer City
  ,state_id       INT  NOT NULL REFERENCES dsci_504.states(state_id) --State ID
  ,cus_zip        TEXT --5-digit zip code
  ,cus_phone      TEXT --10-digit telephone
  ,cus_join_date  DATE --Customer join date
  ,cus_app_cd     TEXT --Customer appreication code (1-7)
  ,cus_app_num    TEXT --Customer loyalty program number
);

CREATE TABLE

Total execution time: 00:00:00.052

In [4]:
-- 3) WAREHOUSES
CREATE TABLE dsci_504.warehouses (
  warehouse_id   SERIAL PRIMARY KEY --Warehouse ID
  ,warehouse_name TEXT  NOT NULL --Full city name of warehouse
  ,state_id       INT   NOT NULL REFERENCES dsci_504.states(state_id) --Integer State ID
);

CREATE TABLE

Total execution time: 00:00:00.014

In [5]:
-- 4) SUPPLIERS
CREATE TABLE dsci_504.suppliers (
  sup_id   SERIAL PRIMARY KEY --Supplier ID number
  ,sup_name TEXT  NOT NULL --Supplier Text Name
  ,sup_ctry TEXT --Supplier Country(s)
);

CREATE TABLE

Total execution time: 00:00:00.013

In [6]:
-- 5) COMPONENTS
CREATE TABLE dsci_504.components (
  comp_id    SERIAL PRIMARY KEY --Component ID Number
  ,comp_name  TEXT  NOT NULL --Name of Component
  ,comp_cost  NUMERIC(10,2) --Cost of Component
  ,sup_id     INT   REFERENCES dsci_504.suppliers(sup_id) --Supplier Integer ID
  ,comp_cat   TEXT --Component category (bucketed)
);

CREATE TABLE

Total execution time: 00:00:00.015

In [7]:
-- 6) BUILDS
CREATE TABLE dsci_504.builds (
  build_id   SERIAL PRIMARY KEY --Build ID
  ,build_name TEXT  NOT NULL --Build name
);

CREATE TABLE

Total execution time: 00:00:00.013

In [8]:
-- 7) BUILD_COMPONENTS (many-to-many Builds ↔ Components)
CREATE TABLE dsci_504.build_components (
  build_id INT NOT NULL REFERENCES dsci_504.builds(build_id) --Integer Build ID
  ,comp_id INT NOT NULL REFERENCES dsci_504.components(comp_id) --Integer Component ID
  ,PRIMARY KEY (build_id, comp_id)
);

CREATE TABLE

Total execution time: 00:00:00.008

In [9]:
-- 8) PRODUCTS
CREATE TABLE dsci_504.products (
  prod_id           SERIAL PRIMARY KEY --Product ID
  ,prod_cat_name     TEXT --Product Category Name
  ,prod_manufacturer TEXT --Product Manufacturer (could be Supplier)
  ,prod_name         TEXT NOT NULL --Product Name
  ,prod_description  TEXT --Product Description
  ,prod_price        NUMERIC(10,2) --Product Price
  ,prod_class        TEXT --Product Classification
  ,country_origin    TEXT --Country of Origin
);

CREATE TABLE

Total execution time: 00:00:00.058

In [10]:
-- 9) PRODUCT_BUILDS (many-to-many Products ↔ Builds)
CREATE TABLE dsci_504.product_builds (
  prod_id  INT NOT NULL REFERENCES dsci_504.products(prod_id) --Integer Product ID
  ,build_id INT NOT NULL REFERENCES dsci_504.builds(build_id) --Integer Build ID
  ,PRIMARY KEY (prod_id, build_id)
);

CREATE TABLE

Total execution time: 00:00:00.010

In [11]:
-- 10) ORDERS (header)
CREATE TABLE dsci_504.orders (
  ord_id        SERIAL PRIMARY KEY --Order ID
  ,ord_date      DATE  NOT NULL --Order Date
  ,ord_tax_loc   INT --Order Tax Location (State)
  ,order_tot     NUMERIC(12,2) --Order Total (all lines)
  ,ord_ship_add  BOOLEAN --Order Ship Address is same as ordering customer
  ,ord_ship_date DATE --Order Ship Date
  ,ord_track_num TEXT --Order Tracking Number
  ,warehouse_id  INT  REFERENCES dsci_504.warehouses(warehouse_id) --Integer Warehouse ID order shipped from
  ,cus_id        INT  NOT NULL REFERENCES dsci_504.customers(cus_id) --Integer Customer ID placing order
);

CREATE TABLE

Total execution time: 00:00:00.014

In [12]:
-- 11) ORDER_ITEMS (detail)
CREATE TABLE dsci_504.order_items (
  ord_id      INT  NOT NULL REFERENCES dsci_504.orders(ord_id) --Integer Order ID
  ,prod_id    INT  NOT NULL REFERENCES dsci_504.products(prod_id) --Integer Product ID
  ,quantity   INT  NOT NULL --Quantity ordered
  ,line_total NUMERIC(12,2) NOT NULL --Total for item in line on order
  ,PRIMARY KEY (ord_id, prod_id)
);

CREATE TABLE

Total execution time: 00:00:00.008

In [13]:
-- 12) RETURNS (header)
CREATE TABLE dsci_504.returns (
  rac_id           SERIAL PRIMARY KEY --Return Authorization ID
  ,ord_id           INT    NOT NULL REFERENCES dsci_504.orders(ord_id) --Integer Order ID
  ,cus_id           INT    NOT NULL REFERENCES dsci_504.customers(cus_id) --Integer Customer ID
  ,warehouse_id     INT    NOT NULL REFERENCES dsci_504.warehouses(warehouse_id) --Integer Warehouse ID
  ,return_date      DATE   -- Return Date (date approved), previously was set to be NOT NULL
  ,tot_ret_item_cnt INT --Number of items returned
  ,tot_ret_amnt     NUMERIC(12,2) --Total of all returned items
);

CREATE TABLE

Total execution time: 00:00:00.016

**Modifcations:**

1\. Removed \`NOT NULL\` constraints on the \`return\_date\` column.

In [14]:
-- 13) RETURN_ITEMS (detail)
CREATE TABLE dsci_504.return_items (
  rac_id        INT     NOT NULL REFERENCES dsci_504.returns(rac_id) --RAC ID
  ,prod_id       INT     REFERENCES dsci_504.products(prod_id) --Integer Product ID
  ,comp_id       INT     REFERENCES dsci_504.components(comp_id) -- Integer Component ID
  ,return_qty    INT --Number of items returned on the line
  ,return_amount NUMERIC(12,2) --Amount of item
  ,return_reason TEXT --Returning reason (binned)
  ,PRIMARY KEY (rac_id, prod_id, comp_id)
);

CREATE TABLE

Total execution time: 00:00:00.013

In [15]:
-- 14) TAXES (lookup)
CREATE TABLE dsci_504.taxes (
  tax_id   SERIAL PRIMARY KEY --Integer ID
  ,tax_desc TEXT --Description of tax location (state)
  ,tax_rate NUMERIC(5,4) --Tax rate in whole percent
);

CREATE TABLE

Total execution time: 00:00:00.059

In [16]:
-- 15) ORDER_TAXES (bridge Orders ↔ Taxes)
CREATE TABLE dsci_504.order_taxes (
  ord_id INT NOT NULL REFERENCES dsci_504.orders(ord_id) --Integer Order ID
  ,ord_date DATE --
  ,tax_id INT NOT NULL REFERENCES dsci_504.taxes(tax_id) --Integer Tax ID
  ,PRIMARY KEY (ord_id, tax_id)
);

CREATE TABLE

Total execution time: 00:00:00.017

**Modifcations:**

1\. Added \`ord\_date\` to the schema as this column is available in the \`order\_items.csv\` file.

2\. Added \`tax\_id\` column to the \`order\_taxes.csv\` file with randomly \`id\` assigned.

3\. Deleted \`tax\_rate\` column from the <span style="color: var(--vscode-foreground);">`order_taxes.csv`&nbsp;</span> <span style="color: var(--vscode-foreground);">file as it creates data redundancy.</span>

**Step 3: Import the data into the OPC tables in PostgreSQL. Use a new code cell for each file.**

In [17]:
--Enter your SQL code here. 
-- 1) Import data into dsci_504.states table
COPY dsci_504.states(state_id, state)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\states.csv'
DELIMITER ','
CSV HEADER;

COPY 31

Total execution time: 00:00:00.003

In [18]:
-- 2) Import data into dsci_504.customers table
COPY dsci_504.customers(cus_id, cus_num, cus_last_name, cus_first_name, cus_add_num, cus_address, 
cus_city, state_id, cus_zip, cus_phone, cus_join_date, cus_app_cd, cus_app_num)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\customers.csv'
DELIMITER ','
CSV HEADER;

COPY 2619

Total execution time: 00:00:00.137

In [19]:
-- 3) Import data into dsci_504.warehouses table
COPY dsci_504.warehouses(warehouse_id, warehouse_name, state_id)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\warehouses.csv'
DELIMITER ','
CSV HEADER;

COPY 3

Total execution time: 00:00:00.030

In [20]:
-- 4) Import data into dsci_504.suppliers table
COPY dsci_504.suppliers(sup_id, sup_name, sup_ctry)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\suppliers.csv'
DELIMITER ','
CSV HEADER;

COPY 72

Total execution time: 00:00:00.005

In [21]:
-- 5) Import data into dsci_504.components table
COPY dsci_504.components(comp_id, comp_name, comp_cost, sup_id, comp_cat)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\components.csv'
DELIMITER ','
CSV HEADER;

COPY 271

Total execution time: 00:00:00.018

In [22]:
-- 6) Import data into dsci_504.builds table
COPY dsci_504.builds(build_id, build_name)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\builds.csv'
DELIMITER ','
CSV HEADER;

COPY 80

Total execution time: 00:00:00.004

In [23]:
-- 7) Import data into dsci_504.build_components table
COPY dsci_504.build_components(build_id, comp_id)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\build_components.csv'
DELIMITER ','
CSV HEADER;

COPY 691

Total execution time: 00:00:00.059

In [24]:
-- 8) Import data into dsci_504.products table
COPY dsci_504.products(prod_id,prod_cat_name,prod_manufacturer,prod_name,prod_description,prod_price,prod_class,country_origin)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\products.csv'
DELIMITER ','
CSV HEADER;

COPY 69

Total execution time: 00:00:00.015

In [25]:
-- 9) Import data into dsci_504.product_builds table
COPY dsci_504.product_builds(prod_id, build_id)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\product_builds.csv'
DELIMITER ','
CSV HEADER;

COPY 69

Total execution time: 00:00:00.016

In [26]:
-- 10) Import data into dsci_504.orders table
COPY dsci_504.orders(ord_id, ord_date, ord_tax_loc, order_tot, ord_ship_add, 
ord_ship_date, ord_track_num, warehouse_id, cus_id)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\orders.csv'
DELIMITER ','
CSV HEADER;

COPY 2599

Total execution time: 00:00:00.404

In [27]:
-- 11) Import data into dsci_504.order_items table
COPY dsci_504.order_items(ord_id, prod_id, quantity, line_total)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\order_items.csv'
DELIMITER ','
CSV HEADER;

COPY 2599

Total execution time: 00:00:00.810

In [28]:
-- 12) Import data into dsci_504.returns table
COPY dsci_504.returns(rac_id, ord_id, cus_id, warehouse_id, return_date, tot_ret_item_cnt, tot_ret_amnt)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\returns.csv'
DELIMITER ','
CSV HEADER;

COPY 1024

Total execution time: 00:00:00.215

**Modifications:**

1\. Inserted \`return\_date\` column in the \`returns.csv\` file

2\. Using find and replace tool in excel, I changed all \`0\` occurences in \`warehouse\_id\` column to \`1\` as there is no \`0\` index in the \`warehouses.csv\` file.

In [29]:
-- 13) Import data into dsci_504.return_items table
COPY dsci_504.return_items(rac_id, prod_id, comp_id, return_qty, return_amount, return_reason)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\return_items.csv'
DELIMITER ','
CSV HEADER;

COPY 1024

Total execution time: 00:00:00.226

Modifications:

1\. Inserted \`retrun\_reason\` column in the \`return\_items.csv\` file.

In [30]:
-- 14) Import data into dsci_504.taxes table
COPY dsci_504.taxes(tax_id, tax_desc, tax_rate)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\taxes.csv'
DELIMITER ','
CSV HEADER;

COPY 31

Total execution time: 00:00:00.009

In [31]:
-- 15) Import data into dsci_504.order_taxes table
COPY dsci_504.order_taxes(ord_id, ord_date, tax_id)
FROM 'C:\PROJECTS\Maryville\DSCI-504\Assignment\Week2\OPC_Data_Tables\order_taxes.csv'
DELIMITER ','
CSV HEADER;

COPY 2599

Total execution time: 00:00:00.577

**Step 4: Execute a simple query on each table to verify the data in each table. Do not worry about missing data at this point. Limit your output to 10 lines using the following query syntax:**

```
SELECT * FROM [schema.table] LIMIT 5;

```

In [32]:
-- 1) Select top 10 records from dsci_504.states table
SELECT *
FROM dsci_504.states
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.010

state_id,state
1,AK
2,AL
3,AZ
4,CA
5,CO
6,CT
7,DE
8,FL
9,GA
10,HI


In [33]:
-- 2) Select top 10 records from dsci_504.customers table
SELECT *
FROM dsci_504.customers
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.009

cus_id,cus_num,cus_last_name,cus_first_name,cus_add_num,cus_address,cus_city,state_id,cus_zip,cus_phone,cus_join_date,cus_app_cd,cus_app_num
2600,NULL,Holzer,Yeni,8617,High Noon Street,Hattiesburg,17,39401,7833472627,2020-11-22,5,OPC85554622
2601,NULL,Hull,Tancredo,7673,Cedat Camp Road,Oswego,19,13126,5154255063,2020-04-19,9,OPC86688258
2602,NULL,Sokol,Teodor,965,Southampton,Salt Lake City,26,84119,8802052764,2020-04-15,14,OPC87710870
2603,NULL,Barnes,Theo,270,High Noon Ave,Coram,19,11727,2779494240,2020-09-14,18,OPC78993136
2604,NULL,Tempest,Simone,751,East Shadow Brook Dr.,Pataskala,20,43062,7692163675,2020-06-24,11,OPC86812641
2605,NULL,Bullock,Pankaj,7524,Van Dyke St.,Southampton,22,18966,2812450867,2020-03-20,3,OPC85939415
2606,NULL,Ware,Reina,656,Grand Ave,Los Angeles,4,90008,3902290668,2020-09-21,6,OPC86472716
2607,NULL,Ray,Rosalin,8564,Old Woodland Rd,Cranston,23,2920,7944432375,2020-06-06,9,OPC87091916
2608,NULL,Stark,Herb,7291,Race St.,Ithaca,19,14850,2978433949,2020-06-27,5,OPC87321099
2609,NULL,Bohme,Bohuslav,135,Homewood Ln,Merrimack,19,3054,3147113467,2020-12-12,19,OPC86853764


In [34]:
-- 3) Select top 10 records from dsci_504.components table
SELECT *
FROM dsci_504.components
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.009

comp_id,comp_name,comp_cost,sup_id,comp_cat
1,Giant Contact SLR,143.00,1,handlebar
2,Rida Alpha R27,136.00,2,handlebar
3,Niner Flat Top,118.00,3,handlebar
4,Enve M7 Carb,69.00,4,handlebar
5,e13 base,91.00,5,handlebar
6,RaceFace NextR Carbon,129.00,6,handlebar
7,OC1,51.00,61,handlebar
8,Kona XC,76.00,8,handlebar
11,Newman Evo,143.00,11,handlebar
12,OC3 Carbon,82.00,61,handlebar


In [35]:
-- 4) Select top 10 records from dsci_504.suppliers table
SELECT *
FROM dsci_504.suppliers
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.010

sup_id,sup_name,sup_ctry
1,Giant,Taiwan
2,Ride Alpha,Andorra
3,Niner,USA
4,Enve,USA
5,e13,USA
6,RaceFace,Canada
8,Kona,USA
9,Deity,USA
10,Whyte,UK
11,Newman,Germany


In [36]:
-- 5) Select top 10 records from dsci_504.components table
SELECT *
FROM dsci_504.components
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.006

comp_id,comp_name,comp_cost,sup_id,comp_cat
1,Giant Contact SLR,143.00,1,handlebar
2,Rida Alpha R27,136.00,2,handlebar
3,Niner Flat Top,118.00,3,handlebar
4,Enve M7 Carb,69.00,4,handlebar
5,e13 base,91.00,5,handlebar
6,RaceFace NextR Carbon,129.00,6,handlebar
7,OC1,51.00,61,handlebar
8,Kona XC,76.00,8,handlebar
11,Newman Evo,143.00,11,handlebar
12,OC3 Carbon,82.00,61,handlebar


In [37]:
-- 6) Select top 10 records from dsci_504.builds table
SELECT *
FROM dsci_504.builds
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.008

build_id,build_name
74,gnar
1,five star
23,mammoth
4,3-star
8,2-star
20,1-star
21,the general
25,atomic
39,nuclear
69,behemoth


In [38]:
-- 7) Select top 10 records from dsci_504.build_components table
SELECT *
FROM dsci_504.build_components
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.010

build_id,comp_id
74,10
74,62
74,100
74,131
74,194
74,278
74,218
74,280
74,279
1,29


In [39]:
-- 8) Select top 10 records from dsci_504.products table
SELECT *
FROM dsci_504.products
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.051

prod_id,prod_cat_name,prod_manufacturer,prod_name,prod_description,prod_price,prod_class,country_origin
9,NULL,63,Tallboy,Full-Suspension Mountain Bike,2799.00,Mid,"2,3,4"
68,NULL,61,Occam,Full-Suspension Mountain Bike,3999.00,Mid,"2,3"
43,NULL,70,Troy,Full-Suspension Mountain Bike,3199.00,Mid,2
37,NULL,22,DV9,Hardtail Mountain Bike,3009.00,Mid,1
26,NULL,8,Process 153,Full-Suspension Mountain Bike,3064.00,Mid,"2,3,4"
69,NULL,61,Occam,Full-Suspension Mountain Bike,3999.00,Mid,"2,3"
27,NULL,8,Hei Hei,Full-Suspension Mountain Bike,4227.99,Mid,"2,3"
38,NULL,22,Ripmo,Full-Suspension Mountain Bike,4399.00,Mid,"2,3,4"
54,NULL,10,G-Series,Full-Suspension Mountain Bike,5656.41,Expert,4
13,NULL,25,SB140,Full-Suspension Mountain Bike,5800.00,Expert,"2,3,4"


In [40]:
-- 9) Select top 10 records from dsci_504.product_builds table
SELECT *
FROM dsci_504.product_builds
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.005

prod_id,build_id
9,51
68,32
43,62
37,19
26,24
69,32
27,22
38,18
54,47
13,41


In [41]:
-- 10) Select top 10 records from dsci_504.orders table
SELECT *
FROM dsci_504.orders
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.008

ord_id,ord_date,ord_tax_loc,order_tot,ord_ship_add,ord_ship_date,ord_track_num,warehouse_id,cus_id
5920416,2018-08-08,6,2338.64,True,2018-08-16,PAC4170462-0,2,27
4825298,2017-03-28,16,5275.00,False,2017-03-30,PAC9343767-0,3,28
7009183,2010-10-14,18,4503.20,False,2010-10-15,PAC762592-3,1,29
5186311,2001-05-12,26,8073.44,False,2001-05-15,PAC6865813-2,1,30
4274782,2002-03-19,25,3572.20,False,2002-03-25,PAC5040387-3,3,31
2579402,2012-11-10,30,4158.96,False,2012-11-11,PAC268985-1,3,32
5974882,2000-07-07,4,3430.93,True,2000-07-10,PAC4112909-1,3,33
6080581,2003-05-22,21,5099.00,True,2003-06-01,PAC2181266-3,2,34
6507619,2010-02-25,7,4199.00,True,2010-03-02,PAC3561253-3,2,35
6921547,2012-02-26,23,5006.53,False,2012-03-06,PAC6665295-3,3,36


In [42]:
-- 11) Select top 10 records from dsci_504.order_items table
SELECT *
FROM dsci_504.order_items
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.005

ord_id,prod_id,quantity,line_total
5920416,57,1,2338.64
4825298,42,1,5275.00
7009183,41,1,4503.20
5186311,15,1,8073.44
4274782,36,1,3572.20
2579402,35,1,4158.96
5974882,43,1,3430.93
6080581,10,1,5099.00
6507619,40,1,4199.00
6921547,49,1,5006.53


In [43]:
-- 12) Select top 10 records from dsci_504.returns table
SELECT *
FROM dsci_504.returns
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.010

rac_id,ord_id,cus_id,warehouse_id,return_date,tot_ret_item_cnt,tot_ret_amnt
178,5484681,1191,1,NULL,0,0.00
179,6727420,1754,1,NULL,2,1008.00
180,4711571,2454,2,NULL,0,0.00
181,3555795,2493,1,NULL,7,970.00
182,5687001,512,1,NULL,0,0.00
183,1985674,930,3,NULL,5,71.00
184,5994079,1504,2,NULL,0,0.00
185,5087861,805,3,NULL,0,0.00
186,6077808,346,1,NULL,0,0.00
187,6241665,2589,2,NULL,9,60.00


In [44]:
-- 13) Select top 10 records from dsci_504.return_items table
SELECT *
FROM dsci_504.return_items
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.016

rac_id,prod_id,comp_id,return_qty,return_amount,return_reason
178,13,41,0,0.00,NULL
179,35,59,2,1008.00,NULL
180,46,48,0,0.00,NULL
181,62,34,7,970.00,NULL
182,46,48,0,0.00,NULL
183,44,46,5,71.00,NULL
184,63,43,0,0.00,NULL
185,28,20,0,0.00,NULL
186,60,57,0,0.00,NULL
187,66,40,9,60.00,NULL


In [45]:
-- 14) Select top 10 records from dsci_504.taxes table
SELECT *
FROM dsci_504.taxes
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.008

tax_id,tax_desc,tax_rate
1,AK,0.0000
2,AL,4.0000
3,AZ,5.6000
4,CA,7.2500
5,CO,7.2500
6,CT,6.3500
7,DE,0.0000
8,FL,6.0000
9,GA,4.0000
10,HI,4.0000


In [46]:
-- 15) Select top 10 records from dsci_504.order_taxes table
SELECT *
FROM dsci_504.order_taxes
LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.010

ord_id,ord_date,tax_id
5920416,2018-08-08,1
4825298,2017-03-28,2
7009183,2010-10-14,3
5186311,2001-05-12,4
4274782,2002-03-19,5
2579402,2012-11-10,6
5974882,2000-07-07,7
6080581,2003-05-22,8
6507619,2010-02-25,9
6921547,2012-02-26,10


**Step 5: Create primary keys within each of the newly created tables to elevate the database to 2NF. Some tables already have unique key columns that simply need to be identified as a Primary Key. (If you did this above when building the tables, simply run the below query for each table).Others may require a new column to do this. Run the below query after each table to validate the contstraints are placed on the tables.**

```

SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = '{schema name}'
             AND rel.relname = '{table name}';

```

In [47]:
-- 1) To validate the constraints are placed on STATES table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'states';

(1 row(s) affected)

Total execution time: 00:00:00.023

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27728,states_pkey,27721,p,False,False,True,27722,0,27727,0,0,,,,True,0,True,[1],NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [48]:
-- 2) To validate the constraints are placed on CUSTOMERS table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'customers';

(2 row(s) affected)

Total execution time: 00:00:00.020

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27737,customers_pkey,27721,p,False,False,True,27730,0,27736,0,0,,,,True,0,True,[1],NULL,NULL,NULL,NULL,NULL,NULL,NULL
27738,customers_state_id_fkey,27721,f,False,False,True,27730,0,27727,0,27722,a,a,s,True,0,True,[8],[1],{96},{96},{96},NULL,NULL,NULL


In [49]:
-- 3) To validate the constraints are placed on WAREHOUSES table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'warehouses';

(2 row(s) affected)

Total execution time: 00:00:00.008

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27751,warehouses_pkey,27721,p,False,False,True,27744,0,27750,0,0,,,,True,0,True,[1],NULL,NULL,NULL,NULL,NULL,NULL,NULL
27752,warehouses_state_id_fkey,27721,f,False,False,True,27744,0,27727,0,27722,a,a,s,True,0,True,[3],[1],{96},{96},{96},NULL,NULL,NULL


In [50]:
-- 4) To validate the constraints are placed on SUPPLIERS table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'suppliers';

(1 row(s) affected)

Total execution time: 00:00:00.013

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27765,suppliers_pkey,27721,p,False,False,True,27758,0,27764,0,0,,,,True,0,True,[1],NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [51]:
-- 5) To validate the constraints are placed on COMPONETNTS table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'components';

(2 row(s) affected)

Total execution time: 00:00:00.005

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27774,components_pkey,27721,p,False,False,True,27767,0,27773,0,0,,,,True,0,True,[1],NULL,NULL,NULL,NULL,NULL,NULL,NULL
27775,components_sup_id_fkey,27721,f,False,False,True,27767,0,27764,0,27758,a,a,s,True,0,True,[4],[1],{96},{96},{96},NULL,NULL,NULL


In [52]:
-- 6) To validate the constraints are placed on BUILDS table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'builds';

(1 row(s) affected)

Total execution time: 00:00:00.006

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27788,builds_pkey,27721,p,False,False,True,27781,0,27787,0,0,,,,True,0,True,[1],NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [53]:
-- 7) To validate the constraints are placed on BUILD_COMPONENTS table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'build_components';

(3 row(s) affected)

Total execution time: 00:00:00.010

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27793,build_components_pkey,27721,p,False,False,True,27789,0,27792,0,0,,,,True,0,True,"[1, 2]",NULL,NULL,NULL,NULL,NULL,NULL,NULL
27794,build_components_build_id_fkey,27721,f,False,False,True,27789,0,27787,0,27781,a,a,s,True,0,True,[1],[1],{96},{96},{96},NULL,NULL,NULL
27799,build_components_comp_id_fkey,27721,f,False,False,True,27789,0,27773,0,27767,a,a,s,True,0,True,[2],[1],{96},{96},{96},NULL,NULL,NULL


In [54]:
-- 8) To validate the constraints are placed on PRODUCTS table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'products';

(1 row(s) affected)

Total execution time: 00:00:00.005

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27812,products_pkey,27721,p,False,False,True,27805,0,27811,0,0,,,,True,0,True,[1],NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [55]:
-- 9) To validate the constraints are placed on PRODUCT_BUILDS table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'product_builds';

(3 row(s) affected)

Total execution time: 00:00:00.007

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27817,product_builds_pkey,27721,p,False,False,True,27813,0,27816,0,0,,,,True,0,True,"[1, 2]",NULL,NULL,NULL,NULL,NULL,NULL,NULL
27818,product_builds_prod_id_fkey,27721,f,False,False,True,27813,0,27811,0,27805,a,a,s,True,0,True,[1],[1],{96},{96},{96},NULL,NULL,NULL
27823,product_builds_build_id_fkey,27721,f,False,False,True,27813,0,27787,0,27781,a,a,s,True,0,True,[2],[1],{96},{96},{96},NULL,NULL,NULL


In [56]:
-- 10) To validate the constraints are placed on ORDERS table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'orders';

(3 row(s) affected)

Total execution time: 00:00:00.007

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27836,orders_pkey,27721,p,False,False,True,27829,0,27835,0,0,,,,True,0,True,[1],NULL,NULL,NULL,NULL,NULL,NULL,NULL
27837,orders_warehouse_id_fkey,27721,f,False,False,True,27829,0,27750,0,27744,a,a,s,True,0,True,[8],[1],{96},{96},{96},NULL,NULL,NULL
27842,orders_cus_id_fkey,27721,f,False,False,True,27829,0,27736,0,27730,a,a,s,True,0,True,[9],[1],{96},{96},{96},NULL,NULL,NULL


In [57]:
-- 11) To validate the constraints are placed on ORDER_ITEMS table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'order_items';

(3 row(s) affected)

Total execution time: 00:00:00.006

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27851,order_items_pkey,27721,p,False,False,True,27847,0,27850,0,0,,,,True,0,True,"[1, 2]",NULL,NULL,NULL,NULL,NULL,NULL,NULL
27852,order_items_ord_id_fkey,27721,f,False,False,True,27847,0,27835,0,27829,a,a,s,True,0,True,[1],[1],{96},{96},{96},NULL,NULL,NULL
27857,order_items_prod_id_fkey,27721,f,False,False,True,27847,0,27811,0,27805,a,a,s,True,0,True,[2],[1],{96},{96},{96},NULL,NULL,NULL


In [58]:
-- 12) To validate the constraints are placed on RETURNS table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'returns';

(4 row(s) affected)

Total execution time: 00:00:00.005

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27868,returns_pkey,27721,p,False,False,True,27863,0,27867,0,0,,,,True,0,True,[1],NULL,NULL,NULL,NULL,NULL,NULL,NULL
27869,returns_ord_id_fkey,27721,f,False,False,True,27863,0,27835,0,27829,a,a,s,True,0,True,[2],[1],{96},{96},{96},NULL,NULL,NULL
27874,returns_cus_id_fkey,27721,f,False,False,True,27863,0,27736,0,27730,a,a,s,True,0,True,[3],[1],{96},{96},{96},NULL,NULL,NULL
27879,returns_warehouse_id_fkey,27721,f,False,False,True,27863,0,27750,0,27744,a,a,s,True,0,True,[4],[1],{96},{96},{96},NULL,NULL,NULL


In [59]:
-- 13) To validate the constraints are placed on RETURN_ITEMS table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'return_items';

(4 row(s) affected)

Total execution time: 00:00:00.007

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27890,return_items_pkey,27721,p,False,False,True,27884,0,27889,0,0,,,,True,0,True,"[1, 2, 3]",NULL,NULL,NULL,NULL,NULL,NULL,NULL
27891,return_items_rac_id_fkey,27721,f,False,False,True,27884,0,27867,0,27863,a,a,s,True,0,True,[1],[1],{96},{96},{96},NULL,NULL,NULL
27896,return_items_prod_id_fkey,27721,f,False,False,True,27884,0,27811,0,27805,a,a,s,True,0,True,[2],[1],{96},{96},{96},NULL,NULL,NULL
27901,return_items_comp_id_fkey,27721,f,False,False,True,27884,0,27773,0,27767,a,a,s,True,0,True,[3],[1],{96},{96},{96},NULL,NULL,NULL


In [60]:
-- 14) To validate the constraints are placed on TAXES table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'taxes';

(1 row(s) affected)

Total execution time: 00:00:00.006

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27914,taxes_pkey,27721,p,False,False,True,27907,0,27913,0,0,,,,True,0,True,[1],NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [61]:
-- 15) To validate the constraints are placed on ORDER_TAXES table
SELECT con.*
    FROM pg_catalog.pg_constraint con
        INNER JOIN pg_catalog.pg_class rel ON rel.oid = con.conrelid
        INNER JOIN pg_catalog.pg_namespace nsp ON nsp.oid = connamespace
        WHERE nsp.nspname = 'dsci_504'
             AND rel.relname = 'order_taxes';

(3 row(s) affected)

Total execution time: 00:00:00.008

oid,conname,connamespace,contype,condeferrable,condeferred,convalidated,conrelid,contypid,conindid,conparentid,confrelid,confupdtype,confdeltype,confmatchtype,conislocal,coninhcount,connoinherit,conkey,confkey,conpfeqop,conppeqop,conffeqop,confdelsetcols,conexclop,conbin
27919,order_taxes_pkey,27721,p,False,False,True,27915,0,27918,0,0,,,,True,0,True,"[1, 3]",NULL,NULL,NULL,NULL,NULL,NULL,NULL
27920,order_taxes_ord_id_fkey,27721,f,False,False,True,27915,0,27835,0,27829,a,a,s,True,0,True,[1],[1],{96},{96},{96},NULL,NULL,NULL
27925,order_taxes_tax_id_fkey,27721,f,False,False,True,27915,0,27913,0,27907,a,a,s,True,0,True,[3],[1],{96},{96},{96},NULL,NULL,NULL


**Step 6: CREATE foreign key associations for each table to elevate the database to 3NF. These should be done within each respective table. If this was completed above, note so and move on to Step 7.**

In [62]:
--Enter your SQL code here

-- This was already done inline during table creation (REFERENCES dsci_504.table_name(id_column))

Total execution time: 00:00:00

**Step 7: Execute two queries of your choosing on two different tables in the database.**

In [63]:
--Enter SQL code here

-- Top five most expensive products
SELECT prod_name, prod_price
FROM dsci_504.products
ORDER BY prod_price DESC
LIMIT 5;

(5 row(s) affected)

Total execution time: 00:00:00.007

prod_name,prod_price
SB150,8000.00
SB160,7699.99
E-Series,7565.66
Wreckoning,7399.00
Rail,7399.00


In [64]:
--Enter SQL code here

-- Number of orders per customer
SELECT c.cus_id, c.cus_first_name, c.cus_last_name, COUNT(o.ord_id) AS total_orders
FROM dsci_504.customers c
JOIN dsci_504.orders o ON c.cus_id = o.cus_id
GROUP BY c.cus_id
ORDER BY total_orders DESC
LIMIT 5;

(5 row(s) affected)

Total execution time: 00:00:00.026

cus_id,cus_first_name,cus_last_name,total_orders
1731,Corey,Harper,2
1728,Thomas,Manning,2
2335,Taylor,Booker,1
652,Reginald,Peters,1
1269,Miguel,Dawson,1


In [ ]:
jupyter nbconvert --to html DSCI504_Wk2_Assignment.ipynb